# Load libraries

In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

%load_ext rpy2.ipython

In [2]:
%%R 
library(tidyverse)
library(cowplot)
library(ggforce)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Load data

In [3]:
%%R -o best_hits

#reads best_hits file from cinful output
best_hits <- read_csv("data/best_hits_10genomes_working_final.csv")

Rows: 7537 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): qseqid, sseqid, component, cinful_id, pephash, sample, contig, seq
dbl (13): pident, length, mismatch, gapopen, qstart, qend, sstart, send, eva...
lgl  (4): hmmerHit, verified, allStandardAA, signalMatch

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
%%R -o best_MFP_hits

#reads best_MFP_candidates from cinful output
best_MFP_hits <- read_csv("data/new_best_MFP_candidates.csv")

Rows: 136 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): cinful_id, pephash, sample, contig, seq, cvab_hit
dbl (3): start, stop, strand
lgl (1): allStandardAA

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


The goal here is to breakdown the assemblies with microcins into the following categories
1. novel
2. pcat
3. mfp
4. immunity

novel is already taken care of from the default cinful output.

For pcat we just look at best hits for CvaB and see if those headers match up with the other hits. For MFP, we will need the separate output file. Then take those headers and determine which PCAT hits also had a MFP hit.

In [5]:
#Creates pcat_hits (subset of best_hits selecting for CvaB.verified)
pcat_hits = best_hits[best_hits["component"] == "CvaB.verified"]

#Display results
pcat_hits.shape[0]

264

In [6]:
#Matches the contig of the PCAT hit to the MFP hit and creates file pcat 
#pcat_hits_MFP = pcat_hits[pcat_hits["contig"].isin(best_MFP_hits["contig"])]

#checks MFP file against microcin (best hits) file 
#mfp_hits = best_hits[best_hits["sample"].isin(best_MFP_hits["sample"])]

mfp_hits = best_hits[best_hits["component"] == "MFP.verified"]

#Display results
mfp_hits.shape[0]

289

In [7]:
#Creates a subset of best_hits only containing the immunity proteins
best_hits_immunity_protein = best_hits[best_hits["component"] == "immunity_proteins.verified"]

#Creates a subset of best_hits only containing the microcins
microcin_hits_all = best_hits[best_hits["component"] == "microcins.verified"]

In [8]:
#Creates column "pcat_hits_assembly" and marks T/F if sample names match between microcin and pcat_hits
microcin_hits_all["pcat_hits_assembly"] = microcin_hits_all["sample"].isin(pcat_hits["sample"])

#Creates column "mfp_hits_assembly" and marks T/F if  samples names match between microcin and pcat_hits_MFP
#microcin_hits_all["mfp_hits_assembly"] = microcin_hits_all["sample"].isin(pcat_hits_MFP["sample"])

#Creates column "mfp_hits_assembly" and marks T/F if  samples names match between microcin and pcat_hits_MFP
microcin_hits_all["mfp_hits_assembly"] = microcin_hits_all["sample"].isin(mfp_hits["sample"])

#Creates column "immunity_contig" and marks T/F if samples names match between microcin and immunity proteins
microcin_hits_all["immunity_contig"] = microcin_hits_all["contig"].isin(best_hits_immunity_protein["contig"])

#Creates column "immunity_genome" and marks T/F if samples names match between microcin and immunity proteins
microcin_hits_all["immunity_genome"] = microcin_hits_all["sample"].isin(best_hits_immunity_protein["sample"])

#Creates a verified subset of microcin_hits using HMMer hits (recorded from cinful) 
microcin_hits_verified = microcin_hits_all[microcin_hits_all["hmmerHit"] == True]

#Display output
microcin_hits_verified



/tmp/ipykernel_64032/280522086.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  microcin_hits_all["pcat_hits_assembly"] = microcin_hits_all["sample"].isin(pcat_hits["sample"])
/tmp/ipykernel_64032/280522086.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  microcin_hits_all["mfp_hits_assembly"] = microcin_hits_all["sample"].isin(mfp_hits["sample"])
/tmp/ipykernel_64032/280522086.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,...,start,stop,strand,allStandardAA,seq,signalMatch,pcat_hits_assembly,mfp_hits_assembly,immunity_contig,immunity_genome
1,v1_PLS_db490abaa1226bcadb3363823b8bfc07164af25...,H47_sp_P62530_MCHB_ECOLX,100.000,75.0,0.0,0.0,23.0,97.0,1.0,75.0,...,5978.0,6271.0,-1.0,1,MGFERFWSGQAGKTEVKYDGVYMREITESQLRYISGAGGAPATSAN...,1,True,True,True,True
2,v1_PLS_db490abaa1226bcadb3363823b8bfc07164af25...,H47_sp_P62530_MCHB_ECOLX,100.000,75.0,0.0,0.0,23.0,97.0,1.0,75.0,...,37.0,330.0,-1.0,1,MGFERFWSGQAGKTEVKYDGVYMREITESQLRYISGAGGAPATSAN...,1,True,True,False,True
3,v1_PLS_db490abaa1226bcadb3363823b8bfc07164af25...,H47_sp_P62530_MCHB_ECOLX,100.000,75.0,0.0,0.0,23.0,97.0,1.0,75.0,...,253.0,546.0,-1.0,1,MGFERFWSGQAGKTEVKYDGVYMREITESQLRYISGAGGAPATSAN...,1,True,True,False,True
4,v1_PLS_db490abaa1226bcadb3363823b8bfc07164af25...,H47_sp_P62530_MCHB_ECOLX,100.000,75.0,0.0,0.0,23.0,97.0,1.0,75.0,...,1302.0,1595.0,1.0,1,MGFERFWSGQAGKTEVKYDGVYMREITESQLRYISGAGGAPATSAN...,1,True,True,True,True
5,v1_PLS_db490abaa1226bcadb3363823b8bfc07164af25...,H47_sp_P62530_MCHB_ECOLX,100.000,75.0,0.0,0.0,23.0,97.0,1.0,75.0,...,1170.0,1463.0,1.0,1,MGFERFWSGQAGKTEVKYDGVYMREITESQLRYISGAGGAPATSAN...,1,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,v1_PLS_f5f4b229944b80fdc79a5870d6bb61db5c15b92...,I47_tr_Q712Q0_Q712Q0_ECOLX,93.750,48.0,3.0,0.0,1.0,48.0,1.0,48.0,...,172.0,330.0,1.0,1,MREISDHMLDFVKGGMNLNGLPSSTNVIDLRGKDMGTYIDANGACW...,0,True,True,True,True
401,v1_PLS_4ab3e15a6dc44de84c9f3fd57f8afd9b6a7f7af...,I47_tr_Q712Q0_Q712Q0_ECOLX,91.228,57.0,5.0,0.0,1.0,57.0,1.0,57.0,...,3012.0,3245.0,-1.0,1,MREISDHMLDFVKRGMNLNGLPSSTNVIDLRGKDMGTYIDANGACW...,0,True,True,False,True
403,v1_PLS_162b4d1d19ac7e08ea0998616fd4a878ff9bbec...,V_sp_P22522_CEAV_ECOLX,100.000,93.0,0.0,0.0,1.0,93.0,1.0,93.0,...,8234.0,8512.0,1.0,1,MRTLTLNELDSVSGGASGRDIAMAIGTLSGQFVAGGIGAAAGGVAG...,1,True,True,False,True
404,v1_PLS_796a75914de7bf03f46239f1d355a033c9e7e1d...,M_tr_Q83TS1_Q83TS1_ECOLX,96.739,92.0,3.0,0.0,1.0,92.0,1.0,92.0,...,7652.0,7930.0,-1.0,1,MRKPSENEIKQISGGDGNDGQAELIAIGLLAGTFISPGFGSIAGAY...,1,True,True,True,True


In [9]:
#Create subset of microcin hits with only sample, contig, and pephash(for microcin)
mc = microcin_hits_verified.loc[:, ('sample', 'contig','pephash')]

#Create subset of immunity protein hits with sample and contig
ip = best_hits_immunity_protein.loc[:, ('sample', 'contig')]

#Merge microcin and immunity protein, ensuring sample and contig hits. pephash(mc) will carry over for matches
#This DF contains samples where the pephash meets immunity standard, therefore if sample and pephash match,
#we know there is an immunity protein ON THIS GENOME for this specific microcin
mc_to_ip_pephash = mc.merge(ip, how = 'inner', on = ['sample', 'contig'])

#Merging the sample and pephash of the verified microcins to the contig match from above
#This results in a df that has sample and pephash verification, matching to a known immunity protein
genome_and_pep_match = microcin_hits_verified.merge(mc_to_ip_pephash, how = 'inner', on = ['sample', 'pephash'])

#Final result creates a new column, genome_provides_immunity, that shows where an immunity protein
#is present on the genome, matches the pephash from the microcin, and provides immunity to that specific microcin
microcin_hits_verified["genome_provides_immunity"] = microcin_hits_verified["sample"].isin(genome_and_pep_match["sample"])

#New column, final_immunity, that includes both genome/pephash matches AND contig matches
microcin_hits_verified["final_immunity"] = (microcin_hits_verified["contig"].isin(best_hits_immunity_protein["contig"]) | microcin_hits_verified["genome_provides_immunity"])

/tmp/ipykernel_64032/1932087489.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  microcin_hits_verified["genome_provides_immunity"] = microcin_hits_verified["sample"].isin(genome_and_pep_match["sample"])
/tmp/ipykernel_64032/1932087489.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  microcin_hits_verified["final_immunity"] = (microcin_hits_verified["contig"].isin(best_hits_immunity_protein["contig"]) | microcin_hits_verified["genome_provides_immunity"])


In [10]:
%%R -i microcin_hits_verified
microcin_hits_verified_melt <- 
microcin_hits_verified %>%
	as.tibble() %>%
	mutate(a_component = case_when(
		component == "microcins.verified" ~ "Microcin Hits"
	)) %>%
	mutate(b_verified = case_when(
		pident < 100 ~ "Novel Microcin",
		pident == 100 ~ "Verified Microcin"
	)) %>%
	mutate(c_pcat_hits_assembly = case_when(
		pcat_hits_assembly == TRUE ~ "Contains PCAT",
		pcat_hits_assembly == FALSE ~ "PCAT Absent"
	)) %>%
	mutate(d_mfp_hits_assembly = case_when(
		mfp_hits_assembly == TRUE ~ "Contains MFP",
		mfp_hits_assembly == FALSE ~ "MFP Absent"
	)) %>%
	mutate(e_immunity_contig = case_when(
		immunity_contig == TRUE ~ "Contains IP",
		immunity_contig == FALSE ~ "IP Absent"
    )) %>%
	group_by(a_component,b_verified, 
	c_pcat_hits_assembly, d_mfp_hits_assembly,
	e_immunity_contig) %>%
	summarise(n = n()) %>%
	reshape2::melt() %>%
	gather_set_data(1:5) %>%
	as.tibble() #%>%

microcin_hits_verified_melt

`summarise()` has grouped output by 'a_component', 'b_verified',
'c_pcat_hits_assembly', 'd_mfp_hits_assembly'. You can override using the
`.groups` argument.


R[write to console]: Using a_component, b_verified, c_pcat_hits_assembly, d_mfp_hits_assembly, e_immunity_contig as id variables



# A tibble: 80 × 10
   a_component   b_verified   c_pcat_hits_ass… d_mfp_hits_asse… e_immunity_cont…
   <chr>         <chr>        <chr>            <chr>            <chr>           
 1 Microcin Hits Novel Micro… Contains PCAT    Contains MFP     Contains IP     
 2 Microcin Hits Novel Micro… Contains PCAT    Contains MFP     IP Absent       
 3 Microcin Hits Novel Micro… Contains PCAT    MFP Absent       Contains IP     
 4 Microcin Hits Novel Micro… Contains PCAT    MFP Absent       IP Absent       
 5 Microcin Hits Novel Micro… PCAT Absent      Contains MFP     Contains IP     
 6 Microcin Hits Novel Micro… PCAT Absent      Contains MFP     IP Absent       
 7 Microcin Hits Novel Micro… PCAT Absent      MFP Absent       Contains IP     
 8 Microcin Hits Novel Micro… PCAT Absent      MFP Absent       IP Absent       
 9 Microcin Hits Verified Mi… Contains PCAT    Contains MFP     Contains IP     
10 Microcin Hits Verified Mi… Contains PCAT    Contains MFP     IP Absent       
# … with

In [11]:
%%R

#Save output in supplemental data
microcin_hits_verified_melt %>%
	write_csv("data/supplemental/Ecoli_microcin_sankey_data.csv")

In [13]:
%%R -w 4500 -h 1200
microcin_hits_verified_melt %>%
	group_by(y) %>%
	summarise(sum_value = sum(value)) %>%
	unite("category", c(y,sum_value), sep = ": ",remove = FALSE) %>%
	full_join(microcin_hits_verified_melt) %>%
	ggplot(aes(x, id = id, split = category, value = value)) +
# Next line "fill = *" determines the color split for the sankey plot
		geom_parallel_sets(aes(fill = b_verified, axis.width = 0.1,
			alpha=0.25
			)) +
		geom_parallel_sets_axes(axis.width = 0.05, 
                                color = "black", 
                                fill = "#BBBBBB") +

		geom_parallel_sets_labels(colour = "black", 
                                  angle = 0, 
                                  size = 28,
                                  position = position_nudge(x = 0.08),  
                                  hjust = 0) +

		scale_fill_manual(values=c("#b6d9e8","#ffd1d1")) +
		theme_nothing()

#Uncomment below to save
ggsave("plots/AF/04_Sankey_AF_rerun.svg", width = 4500, height = 1200, units = "px")


Joining, by = "y"
